# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

In [320]:
import pandas as pd
import numpy as np
import re

In [321]:
df=pd.read_csv("https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv",encoding="utf-8")

In [322]:
df.head()

,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30
3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55


In [323]:
df.describe()

,part,company,country,city,price,date
count,20558,20568,18397,20535,20567,20568
unique,100,35,13,30,19214,20552
top,17156-617,Thoughtmix,Portugal,Amadora\t,-,10/04/2017
freq,300,2795,7383,2787,5,7


In [324]:
df.dtypes

part       object
company    object
country    object
city       object
price      object
date       object
dtype: object

In [325]:
for column in df.columns:
    print("Data type in {} is {}.".format(column,type(df[column][0])))

Data type in part is <class 'str'>.
Data type in company is <class 'str'>.
Data type in country is <class 'str'>.
Data type in city is <class 'str'>.
Data type in price is <class 'str'>.
Data type in date is <class 'str'>.


In [326]:
# check for empty string
categorical=['company','country','city']
for column in categorical:
    print("" in df[column])

False
False
False


In [327]:
# check for NaN
for column in df.columns:
    print("column {} has {} null value".format(column, df[column].isnull().sum().sum()))

column part has 10 null value
column company has 0 null value
column country has 2171 null value
column city has 33 null value
column price has 1 null value
column date has 0 null value


### part column

In [328]:
df.part.nunique()

100

In [329]:
df.part.value_counts(dropna = False)

17156-617     300
37205-992     295
52959-433     293
0268-6107     293
54868-0823    292
62011-0219    291
51346-145     289
13537-259     287
58118-5060    286
10237-832     285
68084-172     285
24385-268     281
51138-049     280
49348-574     280
68462-565     277
52125-136     274
16714-295     272
49349-106     268
68752-024     268
76454-100     265
0699-7041     265
49520-501     265
24286-1562    265
16729-167     264
52343-025     263
49967-724     262
65044-3014    253
0228-2167     249
0603-6134     247
37205-951     247
             ... 
44946-1046    166
68084-595     166
67544-356     166
60505-2867    162
51079-947     161
49035-530     158
54868-6311    155
68180-121     150
51060-032     149
62670-4404    148
13537-468     148
42254-213     141
54973-9150    140
0185-0373     137
36800-952     131
50458-560     130
49349-314     128
54123-957     127
51531-9500    126
59779-601     126
76117-001     124
49288-0285    124
16477-306     117
41167-0843    116
49288-0655

#### handle missing values in `part`

In [330]:
df[df.part.isna()]

,part,company,country,city,price,date
14916,NaN,Yozio,Greece,Patras,518.38€,2018-02-17 21:43:43
14917,NaN,Brainsphere,Portugal,Braga,957.24€,2018-02-17 22:12:24
14918,NaN,Lajo,Greece,Thessaloniki,966.06€,2018-02-17 22:54:49
14919,NaN,Roodel,Portugal,Aranhas,873.65€,2018-02-17 23:36:52
17524,NaN,Yozio,Greece,Patras,627.32€,2018-07-12 03:28:46
17525,NaN,Thoughtmix,Portugal,Amadora\t,825.8€,2018-07-12 05:34:07
17526,NaN,Gabcube,Portugal,Almada,188.31€,2018-07-12 06:49:44
17527,NaN,Buzzbean,Germany,Düsseldorf,429.67€,2018-07-12 07:03:50
17528,NaN,Zoonder,United States,Boston,$521.72,2018-07-12 08:38:56
17529,NaN,Twitterbeat,France,Annecy,1031.46€,2018-07-12 09:48:17


In [331]:
prop_nan_part = len(df[df.part.isnull()])/len(df)
prop_nan_part

0.0004861921431349669

Since the proportion of missing values in the part column is only 0.049% of the entire dataset, we can remove the rows with missing part entries.

In [332]:
df=df.dropna(subset=['part'])

In [333]:
df[df.part.isnull()]

,part,company,country,city,price,date


### city column

#### clean strings in `city`

In [334]:
df.city.unique()

array(['Barcelona', 'Thessaloniki', 'Athens', 'Annecy', 'Paris',
       'Düsseldorf', 'London', 'Braga', 'Nanterre', 'Amadora\t',
       'New York', 'Arnhem', 'Nice', 'Lisbon', 'Amsterdam', 'Porto',
       'Boston', 'Niihama', 'Almada', 'Aranhas', 'Heraklion', 'Amiens',
       'Patras', 'Arcueil', 'Lyon', 'Asaka', 'Champagnole', 'Zürich', nan,
       'Monção', 'Vila Fria'], dtype=object)

Result from `unique()` shows that there is an additional `\t` in the city `Amadora`. Remove the`\t`s:

In [335]:
df.city = df.city.str.replace("\t","")

In [336]:
df.city.unique()

array(['Barcelona', 'Thessaloniki', 'Athens', 'Annecy', 'Paris',
       'Düsseldorf', 'London', 'Braga', 'Nanterre', 'Amadora', 'New York',
       'Arnhem', 'Nice', 'Lisbon', 'Amsterdam', 'Porto', 'Boston',
       'Niihama', 'Almada', 'Aranhas', 'Heraklion', 'Amiens', 'Patras',
       'Arcueil', 'Lyon', 'Asaka', 'Champagnole', 'Zürich', nan, 'Monção',
       'Vila Fria'], dtype=object)

#### handle missing values in `city`

In [337]:
df[df.city.isnull()]

,part,company,country,city,price,date
3136,68462-565,Brainsphere,Portugal,NaN,1057.85€,2016-05-20 08:59:53
3137,54092-515,Shufflebeat,Portugal,NaN,772.92€,2016-05-20 09:55:24
3138,0185-0373,Brainsphere,Portugal,NaN,1012.8€,2016-05-20 10:42:10
12820,50563-113,Ntags,NaN,NaN,507.49€,2017-10-20 04:03:11
12821,55154-5057,Brainsphere,NaN,NaN,421.64€,2017-10-20 13:23:34
12822,36987-1697,Ntags,NaN,NaN,937.08€,2017-10-20 18:25:00
12823,68462-565,Thoughtmix,NaN,NaN,777.58€,2017-10-20 21:21:18
12824,0603-6134,Yozio,NaN,NaN,775.05€,2017-10-20 23:17:59
12825,58118-5060,Yozio,NaN,NaN,157.53€,2017-10-20 23:49:53
12826,51060-032,Thoughtmix,NaN,NaN,176.36€,2017-10-21 00:11:55


In [338]:
prop_nan_city = len(df[df.city.isnull()])/len(df)
prop_nan_city

0.001605214515030645

Since the proportion of missing values in the city column is only 0.16% of the entire dataset, we can remove the rows with missing city entries.
<br>
<br>
Moreover, the table above shows that all but three records have missing values in both country and city. This implies that dropping records that have missing values in the city column does not cause a big loss in information in the country column, since only three such country records are not null.

In [339]:
df=df.dropna(subset=['city'])

In [340]:
df[df.city.isnull()]

,part,company,country,city,price,date


### country column

#### clean strings in `country`

In [341]:
df.country.unique()

array(['Spain', 'Greece', 'France', 'Germany', 'United Kingdom',
       'Portugal', 'United States', 'Netherlands', 'Japan', 'Switzerland',
       nan, 'US', 'Tyskland', 'Portuga'], dtype=object)

From the `unique()` result, we can see that `United States` and `US` are the same and thus should be standardized. 
<br>
Similarly, `Portugal` and `Portuga` should be standardized. 
<br>
Additionally, `Tyskland` means `Germany` in Swedish. Thus they need to be standardized too.

In [342]:
df.country = df.country.replace('US', 'United States')
df.country = df.country.replace('Portuga', 'Portugal')
df.country = df.country.replace('Tyskland', 'Germany')

In [343]:
df.country.unique()

array(['Spain', 'Greece', 'France', 'Germany', 'United Kingdom',
       'Portugal', 'United States', 'Netherlands', 'Japan', 'Switzerland',
       nan], dtype=object)

#### handle missing values in  `country` 

In [344]:
df[df.country.isnull()]

,part,company,country,city,price,date
2528,52380-1102,Teklist,NaN,Arnhem,357.78,2016-04-21 04:07:31
2947,52125-136,Brainsphere,NaN,Braga,493.94€,2016-05-10 11:13:43
2956,16714-295,Teklist,NaN,Arnhem,624.4€,2016-05-10 21:57:15
2994,76335-006,Buzzbean,NaN,Düsseldorf,355.24€,2016-05-12 15:17:37
3948,54473-578,Brainsphere,NaN,Braga,123.32€,2016-07-01 01:09:40
3949,54973-9150,Twitterbeat,NaN,Annecy,893.86€,2016-07-01 02:59:50
3950,41167-0843,Brainsphere,NaN,Braga,93.93€,2016-07-01 04:03:16
3951,51060-032,Riffpath,NaN,Heraklion,665.09€,2016-07-01 04:56:16
3952,37205-951,Buzzbean,NaN,Düsseldorf,257.66€,2016-07-01 06:02:32
3953,37205-951,Chatterbridge,NaN,Barcelona,181.97€,2016-07-01 06:26:37


In [345]:
prop_nan_country = len(df[df.country.isnull()])/len(df)
prop_nan_country

0.10431181485992692

Since the fraction of missing values in the country column is 10.43% of the entire dataset which is considerably large, we cannot simply remove the rows with missing countries entries as doing so will result in much information being lost.
<br>
<br>
Instead, we can deduce some of the country names from their corresponding cities.

In [346]:
df.groupby(['city'])['country'].value_counts(dropna = False)

city          country       
Almada        Portugal           323
              NaN                 33
Amadora       Portugal          2511
              NaN                275
Amiens        France             452
              NaN                 46
Amsterdam     Netherlands         40
              NaN                  4
Annecy        France            2037
              NaN                229
Aranhas       Portugal           553
              NaN                 73
Arcueil       France             180
              NaN                 24
Arnhem        Netherlands        387
              NaN                 39
Asaka         Japan               47
              NaN                  7
Athens        Greece             713
              NaN                 99
Barcelona     Spain             1425
              NaN                164
Boston        United States      398
              NaN                 55
Braga         Portugal          1114
              NaN                122
Champagno

In [347]:
city_country_mapping = df.groupby(['city'])['country'].apply(lambda grp: list(grp.value_counts().index)[0]).to_dict()
city_country_mapping

{'Almada': 'Portugal',
 'Amadora': 'Portugal',
 'Amiens': 'France',
 'Amsterdam': 'Netherlands',
 'Annecy': 'France',
 'Aranhas': 'Portugal',
 'Arcueil': 'France',
 'Arnhem': 'Netherlands',
 'Asaka': 'Japan',
 'Athens': 'Greece',
 'Barcelona': 'Spain',
 'Boston': 'United States',
 'Braga': 'Portugal',
 'Champagnole': 'France',
 'Düsseldorf': 'Germany',
 'Heraklion': 'Greece',
 'Lisbon': 'Portugal',
 'London': 'United Kingdom',
 'Lyon': 'France',
 'Monção': 'Portugal',
 'Nanterre': 'France',
 'New York': 'United States',
 'Nice': 'France',
 'Niihama': 'Japan',
 'Paris': 'France',
 'Patras': 'Greece',
 'Porto': 'Portugal',
 'Thessaloniki': 'Greece',
 'Vila Fria': 'Portugal',
 'Zürich': 'Switzerland'}

In [348]:
df[df.country.isnull()].index

Int64Index([ 2528,  2947,  2956,  2994,  3948,  3949,  3950,  3951,  3952,
             3953,
            ...
             9990,  9991, 10950, 11220, 12263, 12543, 12545, 12978, 13721,
            14438],
           dtype='int64', length=2141)

In [349]:
def impute_countries(row):
    return city_country_mapping[row.city]

In [350]:
df.country = df.apply(impute_countries, axis = 1)

In [351]:
df[df.country.isna()]

,part,company,country,city,price,date


### company column

#### clean strings in `company`

In [352]:
df.company.unique()

array(['Chatterbridge', 'Lajo', 'Flipstorm', 'Twitterbeat', 'Voomm',
       'Buzzbean', 'Zooxo', 'Brainsphere', 'Thoughtmix', 'Wordify',
       'Teklist', 'Avaveo', 'Ntags', 'Innojam', 'Shufflebeat', 'Zoonder',
       'Kanoodle', 'Gabcube', 'Roodel', 'Riffpath', 'Eimbee', 'Yozio',
       'Rhycero', 'Realpoint', 'Gabtune', 'Bubblemix', 'Gevee', 'Tagtune',
       'Zooxo.', 'Laj0', 'Ntagz', ' -', ' a', 'aa', 'Thoughtmixz'],
      dtype=object)

In [353]:
df.company.value_counts(dropna=False)

Thoughtmix       2786
Twitterbeat      2266
Zooxo            1667
Chatterbridge    1589
Shufflebeat      1550
Ntags            1508
Buzzbean         1253
Brainsphere      1237
Flipstorm        1193
Wordify           966
Yozio             651
Roodel            626
Eimbee            498
Zoonder           452
Teklist           426
Gabcube           356
Voomm             250
Lajo              219
Avaveo            212
Rhycero           204
Realpoint         158
Riffpath          151
Kanoodle          126
Bubblemix          54
Innojam            44
Gevee              36
Gabtune            27
Tagtune            12
Zooxo.              2
Laj0                1
Thoughtmixz         1
 a                  1
aa                  1
Ntagz               1
 -                  1
Name: company, dtype: int64

From the above result, it can be seen that some erroneous company names/poorly formatted company names. These include:
* ` a`,`aa`,` -` each has occurred once in the dataset. They are likely to be the erroneous entries. Simply remove them.
* `Laj0` should be standardized to `Lajo`. Likewise, `Zooxo.` should be standardized to `Zooxo`.

In [354]:
df.company = df.company.replace([' a','aa',' -'], '')
df=df[df.company != '']

In [355]:
df.company = df.company.replace('Zooxo.','Zooxo')
df.company = df.company.replace('Laj0','Lajo')

In [356]:
df.company.value_counts(dropna=False)

Thoughtmix       2786
Twitterbeat      2266
Zooxo            1669
Chatterbridge    1589
Shufflebeat      1550
Ntags            1508
Buzzbean         1253
Brainsphere      1237
Flipstorm        1193
Wordify           966
Yozio             651
Roodel            626
Eimbee            498
Zoonder           452
Teklist           426
Gabcube           356
Voomm             250
Lajo              220
Avaveo            212
Rhycero           204
Realpoint         158
Riffpath          151
Kanoodle          126
Bubblemix          54
Innojam            44
Gevee              36
Gabtune            27
Tagtune            12
Thoughtmixz         1
Ntagz               1
Name: company, dtype: int64

### date column

In [357]:
# string to datetime object
df['parsed_date'] = pd.to_datetime(df['date'],errors='coerce', infer_datetime_format=True)

In [358]:
# checking
df[df.parsed_date.isnull()]

,part,company,country,city,price,date,parsed_date
3539,17156-617,Thoughtmix,Portugal,Amadora,791.86€,2016-06-32 07:22:28,NaT
3540,54868-5165,Shufflebeat,Portugal,Porto,525.24€,2016-06-32 08:08:48,NaT


In [359]:
df = df.dropna()

### price column

In [360]:
# original intention was to find all currency signs present
# but it turned out that there are other poorly formatted cells other than currency signs
res=[]
for p in df.price:
    for c in p:
        if c.isdigit() == False:
            res.append(c)
x=np.array(res)
np.unique(x)

array(['$', '-', '.', 'a', 'd', 'i', 'n', 'o', 'v', '£', '¥', '€'],
      dtype='<U1')

In [361]:
# find what are those cells
res=[]
for p in df.price:
    for c in p:
        if c in ['a', 'd', 'i', 'n', 'o', 'v']:
            print(p)
            break

void
void
na
na
na


In [362]:
print(len(df))
df=df[df.price != 'void']
df=df[df.price != 'na']
print(len(df))

20519
20514


In [363]:
def parse_price(string):
    amount=re.match(r'([^-\d\.]?)(-?\d*\.?\d*)([\D]?)', str(string))
    lst=[i for i in amount.groups() if i != '']
#     if len(lst) != 2:
#         return float('NaN')
    try:
        res=float(lst[0])
    except ValueError:
        res=float(lst[1])
    return res

In [364]:
def parse_symbol(string):
    amount=re.match(r'([^-\d\.]?)(-?\d*\.?\d*)([\D]?)', str(string))
    lst=[i for i in amount.groups() if i != '']
#     if len(lst) != 2:
#         return float('NaN')
    try:
        number=float(lst[0])
        sign=lst[1]
    except ValueError:
        sign=lst[0]
    return sign

In [365]:
def find_currency(string):
    currency=re.findall("[€$¥£]",string)
    if currency:
        symbol=currency[0]
        if symbol == "€":
            return "Euro"
        elif symbol == "$":
            return "USD"
        elif symbol == "¥":
            return "RMB"
        elif symbol == "£":
            return "Pound"
    else:
        print("error: can't find currenct symbol in {}".format(string))

In [366]:
df["currency"]=df.price.apply(find_currency)

error: can't find currenct symbol in 465.6
error: can't find currenct symbol in 1266.68
error: can't find currenct symbol in 829.3
error: can't find currenct symbol in 357.78
error: can't find currenct symbol in -
error: can't find currenct symbol in -
error: can't find currenct symbol in -
error: can't find currenct symbol in -
error: can't find currenct symbol in -


In [367]:
df=df[df['price'] != "465.6"]
df=df[df['price'] != "1266.68"]
df=df[df['price'] != "829.3"]
df=df[df['price'] != "357.78"]
df=df[df['price'] != "-"]

In [368]:
df['price_amount']=df.price.apply(parse_price)

In [369]:
df['price_symbol']=df.price.apply(parse_symbol)

In [370]:
df.tail()

,part,company,country,city,price,date,parsed_date,currency,price_amount,price_symbol
20563,21695-267,Roodel,Portugal,Aranhas,606.37€,2018-12-31 20:48:14,2018-12-31 20:48:14,Euro,606.37,€
20564,49999-737,Thoughtmix,Portugal,Amadora,828.37€,2018-12-31 21:24:17,2018-12-31 21:24:17,Euro,828.37,€
20565,52343-025,Brainsphere,Portugal,Braga,686.1€,2018-12-31 21:38:02,2018-12-31 21:38:02,Euro,686.10,€
20566,49288-0285,Flipstorm,Greece,Athens,1051.08€,2019-02-21 20:05:00,2019-02-21 20:05:00,Euro,1051.08,€
20567,43419-018,Flipstorm,Greece,Athens,139.56€,2019-05-14 22:48:39,2019-05-14 22:48:39,Euro,139.56,€


In [371]:
df.isnull().sum().sum()

0

---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---